In [5]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from time import sleep
from login import EMAIL
from login import PASSWORD
import re

In [6]:
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10)
driver.maximize_window()

driver.get("https://pro.urbanize.city/users/sign_in")

wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))


iframes = driver.find_elements(By.TAG_NAME, "iframe")
if iframes:
    driver.switch_to.frame(iframes[0])

email_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='email'], input[type='email']")))
email_input.send_keys(EMAIL)

password_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='password'], input[type='password']")))
password_input.send_keys(PASSWORD)

login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.prime[actionlogin]")))
login_button.click()


wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

driver.switch_to.default_content()
wait.until(EC.visibility_of_element_located((By.XPATH, "//a[contains(@class, 'border-cyan-500') and contains(text(), 'Dashboard')]")))


<selenium.webdriver.remote.webelement.WebElement (session="9147ebbe-04cb-4492-9b66-7e5b11b21940", element="afc428e7-3e89-4d64-a89f-6f04d01e7109")>

In [7]:
driver.get("https://pro.urbanize.city/los_angeles/projects")

project_links = driver.find_elements(By.CSS_SELECTOR, "ul.grid a[href^='/los_angeles/projects/']")
page_urls = [link.get_attribute('href') for link in project_links]
project_data = {url: {} for url in page_urls}


for url in page_urls:
  driver.get(url)

  wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
  
  project_name = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1.text-5xl.font-semibold.text-gray-900.tracking-tight.mb-2"))).text
  project_data[url]['project_name'] = project_name

  grid_exists = len(driver.find_elements(By.CSS_SELECTOR, "ul.grid")) > 0

  if grid_exists:
      grid_links = driver.find_elements(By.CSS_SELECTOR, "ul.grid li a")
      grid_urls = [link.get_attribute('href') for link in grid_links]
      project_data[url]['grid_urls'] = grid_urls
      for article_url in grid_urls:
        driver.get(article_url)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

      



In [8]:
def get_project_status(driver):

    return "Status not found"

In [15]:
def pull_data(url):
  driver.get(url)

  #get project name
  wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
  project_name = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1.text-5xl.font-semibold.text-gray-900.tracking-tight.mb-2"))).text

  print(f"Project Name: {project_name}")
  

  #get project status
  status = None
  status_elements = driver.find_elements(By.CSS_SELECTOR, "div.flex.space-x-1.w-full > div")
  for element in status_elements:
    bg_div = element.find_element(By.CSS_SELECTOR, "div[class*='w-full'][class*='h-2.5']")
    if "bg-cyan-600" in bg_div.get_attribute("class"):
      status = element.find_element(By.CSS_SELECTOR, "div.text-sm").text

  print(f"Project Status: {status}")


  #get location
  location_header = driver.find_element(By.XPATH, "//h3[contains(@class, 'text-xl') and contains(text(), 'Location')]")
  location_html = location_header.find_element(By.XPATH, "following-sibling::p").get_attribute('innerHTML')
  # location_text = location_element2.text.replace('\n', ', ')
  location_text = re.sub('<[^<]+?>', '', location_html).replace('\n', '').strip()
  # location_text = ', '.join(part.strip() for part in location_text.split(','))

  print(f"Project Location: {location_text}")


  # #get companies
  # architects_div = driver.find_element(By.CSS_SELECTOR, "div.space-y-1.grow")
  # architect_elements = architects_div.find_elements(By.CSS_SELECTOR, "div.text-cyan-600.font-semibold a")
  # architects = [element.text for element in architect_elements]

  # print(f"Architects: {architects}")


  architects_header = driver.find_element(By.XPATH, "//div[contains(@class, 'text-slate-900') and contains(., 'Architects')]")
  architects_div = architects_header.find_element(By.XPATH, "following-sibling::div[contains(@class, 'space-y-1')]")
  architects_elements = architects_div.find_elements(By.CSS_SELECTOR, "div.text-cyan-600.font-semibold a")
  architects = [element.text for element in architects_elements]
  print(f"Architects: {architects}")

  developers_header = driver.find_element(By.XPATH, "//div[contains(@class, 'text-slate-900') and contains(., 'Developer')]")
  developers_div = developers_header.find_element(By.XPATH, "following-sibling::div[contains(@class, 'space-y-1')]")
  developers_elements = developers_div.find_elements(By.CSS_SELECTOR, "div.text-cyan-600.font-semibold a")
  developers = [element.text for element in developers_elements]
  print(f"Developers: {developers}")


  project_info = {}
  try:
        information_div = driver.find_element(By.XPATH, "//h3[contains(@class, 'text-xl') and contains(text(), 'Information')]/ancestor::div[contains(@class, '')]")
        info_items = information_div.find_elements(By.XPATH, ".//li[contains(@class, 'py-4')]")
        
        for item in info_items:
            label = item.find_element(By.XPATH, ".//div[contains(@class, 'text-slate-900') and contains(@class, 'font-medium')]").text
            value_div = item.find_element(By.XPATH, ".//div[contains(@class, 'text-slate-800') and contains(@class, 'font-normal')]")
            
            if label == "Project uses":
                uses = [use.text for use in value_div.find_elements(By.XPATH, ".//div/div")]
                project_info[label] = uses
                print(f"Project uses: {', '.join(uses)}")
            else:
                value = value_div.text.strip()
                project_info[label] = value
                print(f"{label}: {value}")
  except:
        print("Information section not found or could not be parsed")

    # Return the extracted data as a dictionary
  result = {
        "project_name": project_name,
        "status": status,
        "location": location_text,
        "architects": architects,
        "developers": developers,
        **project_info
    }
    
  print(result)





pull_data('https://pro.urbanize.city/los_angeles/projects/d506b9d2/2143-e-violet-street')

Project Name: 2143 E Violet Street
Project Status: Proposed
Project Location: 2143 Violet St    Los Angeles, CA 90021
Architects: ['Arno Matis Architecture', 'Enns Gauthier', 'Maud Architects', 'Solomon Cordwell Buenz']
Developers: ['Onni Group']
Project uses: Apartments, Retail, Parking
Floors (above ground): 30
Height (ft): 344
Podium: No
# of apartments: 347
Construction type: Type 1 2
Parking spaces: 798
Office space (sqft): 187,000
Retail space (sqft): 200
Information section not found or could not be parsed
{'project_name': '2143 E Violet Street', 'status': 'Proposed', 'location': '2143 Violet St    Los Angeles, CA 90021', 'architects': ['Arno Matis Architecture', 'Enns Gauthier', 'Maud Architects', 'Solomon Cordwell Buenz'], 'developers': ['Onni Group'], 'Project uses': ['Apartments', 'Retail', 'Parking'], 'Floors (above ground)': '30', 'Height (ft)': '344', 'Podium': 'No', '# of apartments': '347', 'Construction type': 'Type 1 2', 'Parking spaces': '798', 'Office space (sqft)':

In [10]:
project_data

{'https://pro.urbanize.city/los_angeles/projects/7230d333/fourth-clover-apartments': {'project_name': 'Fourth Clover Apartments',
  'grid_urls': []},
 'https://pro.urbanize.city/los_angeles/projects/25137888/fairmont-breakers': {'project_name': 'Fairmont Breakers',
  'grid_urls': ['https://la.urbanize.city/post/fairmont-breakers-hotel-debut-long-beach-november-7',
   'https://la.urbanize.city/post/long-beachs-breakers-hotel-reopen-2021',
   'https://la.urbanize.city/post/details-emerge-proposed-revival-long-beachs-breakers-hotel']},
 'https://pro.urbanize.city/los_angeles/projects/c4bf6ab2/madison-place': {'project_name': 'Madison Place',
  'grid_urls': ['https://la.urbanize.city/post/new-look-proposed-development-pasadena-playhouse-district',
   'https://la.urbanize.city/post/mixed-use-development-replace-parking-lot-pasadenas-playhouse-district']},
 'https://pro.urbanize.city/los_angeles/projects/cee313b0/2434-birkdale-avenue': {'project_name': '2434 Birkdale Avenue',
  'grid_urls': 